<a href="https://colab.research.google.com/github/leejuhyoung11/Repair_road/blob/master/Repair_Road.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)
root_path = 'gdrive/My Drive/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


### VGGNet.py

In [0]:
import torch
import torch.nn as nn

import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
import torchvision.models.vgg as vgg



device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

cfg = [32,32,'M', 64,64,128,128,128,'M',256,256,256,512,512,512,'M'] #13 + 3 =vgg16

    
class VGG(nn.Module):
    def __init__(self, features, num_classes=1000, init_weights=True):
        super(VGG, self).__init__()
        
        self.features = features #convolution
        
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),  
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )#FC layer
        
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x) #Convolution 
        x = self.avgpool(x) # avgpool
        x = x.view(x.size(0), -1) #
        x = self.classifier(x) #FC layer
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')  
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)  
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 200
    
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2, padding = 1)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
                     
    return nn.Sequential(*layers)



vgg = make_layers(cfg)



vgg16 = VGG(vgg, 10, True).to(device)





### Alexnet.py

In [0]:
import torch
import torch.nn as nn


class AlexNet(nn.Module):

    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(200, 32, kernel_size=5, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding = 1),
            nn.Conv2d(32, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding = 1),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding = 1),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


### learn.py

In [0]:
import glob
import numpy as np
import cv2
from scipy.ndimage.interpolation import zoom

class Learn(object):
    def __init__(self):
        self.images = []  # 이미지 데이터를 담아둘 배열 생성

    def show_images(self):  # 이미지 데이터가 정상적으로 읽어지는지 보여준다
        for i in self.images:
            cv2.imshow('images', i[0])  # 윈도우에 사진 띄우기
            cv2.waitKey(8)  # 윈도우 대기 시간
        cv2.destroyAllWindows()  # 윈도우 종료

    def import_label(self, label_path):  # 라벨이 들어있는 파일을 읽어와 라벨 배열에 담아준다
        file = open(label_path)  # 라벨파일을 열음
        file = file.read().split()  # 라벨파일을 스플릿해서 읽음
        label_array=[]  # 라벨을 담아줄 배열 생성
        for label in file:
            label_array.append(label)  # 스플릿한 라벨을 배열에 넣어줌
        return label_array

    def import_label_forColab(self, label_path):  # 라벨이 들어있는 파일을 읽어와 라벨 배열에 담아준다
        # file = open(label_path) # 라벨파일을 열음
        # file = file.read().split() # 라벨파일을 스플릿해서 읽음
        label_array = ['clean_road', 'cracked_road', 'perforated_road']  # 라벨을 담아줄 배열 생성
        # for label in file:
        #    label_array.append(label) # 스플릿한 라벨을 배열에 넣어줌
        return label_array

    def find_index(self, label_arr, key):  # 라벨명의 인덱스로 라벨링을 하기 위해 인덱스를 찾아준다
        for index, value in enumerate(label_arr):
            if value == key:
                return index
            else:
              print(value)

    def import_dataset(self, train_path, label_path):  # 이미지를 불러와서 제너럴 데이터폼으로 바꾼다
        print('image loading...')
        label = self.import_label(label_path)  # 라벨을 불러와서 배열에 담음
        for image_data in glob.glob(train_path):  # 절대경로 안에 있는 모든 이미지를 불러옴
            # print(image_data)

            try:  # 불량 데이터를 거르기위한 트라이 익셉트
                image = cv2.resize(cv2.imread(image_data)/255, dsize=(200, 200))  # 불러온 이미지를 0~1사이값으로 바꾼뒤 리사이즈함
                #rotate30 = cv2.getRotationMatrix2D((100, 100), 30, 1)  # 이미지를 부풀리기위해 로테이트해줌
                #rotate45 = cv2.getRotationMatrix2D((100, 100), 45, 1)
                # move = np.float32([1, 0, np.random.randint(100)], [0, 1, np.random.randint(100)])  # error;;;
                #image2 = cv2.warpAffine(image, rotate30, (200, 200))  # 로테이트한 이미지 생성
                #image3 = cv2.warpAffine(image, rotate45, (200, 200))
                #image4 = cv2.flip(image, 1)  # 좌우반전
                #image5 = cv2.flip(image, 1)  # 상하반전
                # image6 = cv2.warpAffine(image, move, (200, 200))
            except:
                print('please remove image : ', image_data)

            self.images.append((image, self.find_index(label, image_data.split('\\')[-2])))  # 위에서 생성한 배열에 텐서로 변환해서 데이터를 담아줌
            #self.images.append((image2, self.find_index(label, image_data.split('\\')[-2])))  # 로테이트한 데이터도 z담아줌
            #self.images.append((image3, self.find_index(label, image_data.split('\\')[-2])))
            #self.images.append((image4, self.find_index(label, image_data.split('\\')[-2])))  # 반전한 데이터도 담아줌
            #self.images.append((image5, self.find_index(label, image_data.split('\\')[-2])))
            # self.images.append((image6, self.find_index(label, image_data.split('\\')[-2])))
        return self.images


    def import_dataset_forColab(self, train_path, label_path):  # 이미지를 불러와서 제너럴 데이터폼으로 바꾼다
        label = self.import_label_forColab(label_path)  # 라벨을 불러와서 배열에 담음
        for image_data in glob.glob(train_path):  # 절대경로 안에 있는 모든 이미지를 불러옴
            images = cv2.imread(image_data)
            try:  # 불량 데이터를 거르기위한 트라이 익셉트
                image = cv2.resize(cv2.imread(image_data) / 255, dsize=(200, 200))  # 불러온 이미지를 0~1사이값으로 바꾼뒤 리사이즈함

            except:
                print('please remove image : ', image_data)

            self.images.append(
                (image, self.find_index(label, image_data.split('/')[-2])))  # 위에서 생성한 배열에 텐서로 변환해서 데이터를 담아줌

        return self.images

   


### main.py

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
#from AlexNet import AlexNet as Net
#from learn import Machine_Learning as Learn

# define
batch_size = 32  # 배치 사이즈
number_of_epoch = 1000  # 에포크 반복 횟수
#### Pycharm ####
# path = 'C:/Users/gjb26/OneDrive/AIprj/prj'
# train_path = path + '/train/*/*'  # 이미지 데이터셋이 들어있는 절대경로
# test_path = path + '/test/*/*'  # 이미지 데이터셋이 들어있는 절대경로
# label_path = './label.txt'  # 라벨파일의 상대경로

#### colab ####
train_path = '/content/gdrive/My Drive/road/train/*/*' # 이미지 데이터셋이 들어있는 절대경로
test_path = '/content/gdrive/My Drive/road/test/*/*' # 이미지 데이터셋이 들어있는 절대경로
label_path = '/content/gdrive/My Drive/road/label.txt' # 라벨파일의 상대경로


if __name__=="__main__":
    training = Learn()  # 객체 생성
    test = Learn()

    #### pycharm ####
    # training.import_dataset(train_path, label_path)  # 데이터셋을 불러옴
    # test.import_dataset(test_path, label_path)

    #### colab ####
    training.import_dataset_forColab(train_path, label_path)
    test.import_dataset_forColab(test_path, label_path)

    # learn.show_images() # 데이터셋이 잘 읽어지는지 확인
    # print(learn.images)

    train_dataset = training.images
    test_dataset = test.images

    # batch size : 32, shffle 사용을 통해 임의의 data 선택
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    is_cuda = False
    if torch.cuda.is_available():  # GPU가 사용가능한지 확인
        is_cuda = True  # GPU가 사용 가능하면 학습을 GPU로함

    #model = VGG()
    #model = model.to(device)
    if is_cuda:
        model.cuda()


    ###  AlexNet ###
    # criterion = nn.CrossEntropyLoss()
    # # optimizer = optim.SGD(model.parameters(), lr=0.01)
    # optimizer = torch.optim.Adam(model.parameters(), lr = 0.1)
    # lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9) 

    # for epoch in range(350):  # loop over the dataset multiple times
    #     running_loss = 0.0
    #     lr_sche.step()

    #     for i, (data, labels) in enumerate(train_loader, 0):
    #         # get the inputs

    #         inputs, labels = data.cuda(), labels.cuda()
    #         # print(inputs)

    #         # zero the parameter graients
    #         optimizer.zero_grad()

    #         # forward + backward + optimize
    #         outputs = model(inputs.float())
    #         loss = criterion(outputs, labels)
    #         loss.backward()
    #         optimizer.step()

    #         # print statistics
    #         running_loss += loss.item()
    #         # if i % 10 == 9:  # 1000 batch 마다 loss 출력
    #     print('[%d, %5d] loss: %.10f' % (epoch + 1, (i + 1), running_loss / 1000))
    #     running_loss = 0.0

    # print('Finished Training')

    # # performance on the whole test dataset
    # correct = 0
    # total = 0
    # clean = 0
    # clean_total = 0
    # cracked = 0
    # cracked_total = 0
    # perforated = 0
    # perforated_total = 0
    # classes = [0, 0, 0]
    # bingo = [0, 0, 0]

    # with torch.no_grad():
    #     for (data, labels) in test_loader:
    #         images, labels = Variable(data.cuda()), Variable(labels.cuda())
    #         outputs = model(images.float())
    #         _, predicted = torch.max(outputs.data, 1)
    #         total += labels.size(0)
    #         correct += (predicted == labels).sum().item()
    #         # print(labels[1].item())
    #         for i in range(batch_size):  # 종류별로 정확도를 출력하기 위한 반복문
    #             try:
    #                 label = labels[i]  # 라벨과 예측값을 인덱스로 비교
    #                 pred = predicted[i]
    #                 # print(label.item())
    #                 if label.item() == 0:
    #                     clean_total += 1
    #                     if pred.item() == 0:
    #                         clean += 1
    #                 elif label.item() == 1:
    #                     cracked_total += 1
    #                     if pred.item() == 1:
    #                         cracked += 1
    #                 elif label.item() == 2:
    #                     perforated_total += 1
    #                     if pred.item() == 2:
    #                         perforated += 1
    #             except:
    #                 print('index error')  # 아직 인덱스를 동적으로 처리하지 못해서 트라이 캐치로 커버함
    # print('Accuracy of clean : {}%'.format(100 * clean / clean_total))
    # print('Accuracy of cracked : {}%'.format(100 * cracked / cracked_total))
    # print('Accuracy of perforated : {}%'.format(100 * perforated / perforated_total))
    # print('Accuracy of test images: %d %%' % (100 * correct / total))



    ###  VGGNet  ###
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.SGD(vgg16.parameters(), lr = 0.005,momentum=0.9)
    lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)
    
    for epoch in range(350):  # loop over the dataset multiple times
        running_loss = 0.0
        lr_sche.step()
        for i, (data, labels) in enumerate(train_loader, 0):
            # get the inputs

            inputs, labels = data.cuda(), labels.cuda()
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter graients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = vgg16(inputs.float())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            # if i % 10 == 9:  # 1000 batch 마다 loss 출력
        print('[%d, %5d] loss: %.10f' % (epoch + 1, (i + 1), running_loss / 1000))
        running_loss = 0.0

    print('Finished Training')
    
    # performance on the whole test dataset
    correct = 0
    total = 0
    clean = 0
    clean_total = 0
    cracked = 0
    cracked_total = 0
    perforated = 0
    perforated_total = 0
    classes = [0, 0, 0]
    bingo = [0, 0, 0]

    with torch.no_grad():
        for (data, labels) in test_loader:
            images, labels = Variable(data.cuda()), Variable(labels.cuda())
            images = images.to(device)
            labels = labels.to(device)
            outputs = vgg16(images.float())
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            # print(labels[1].item())
            for i in range(batch_size):  # 종류별로 정확도를 출력하기 위한 반복문
                try:
                    label = labels[i]  # 라벨과 예측값을 인덱스로 비교
                    pred = predicted[i]
                    # print(label.item())
                    if label.item() == 0:
                        clean_total += 1
                        if pred.item() == 0:
                            clean += 1
                    elif label.item() == 1:
                        cracked_total += 1
                        if pred.item() == 1:
                            cracked += 1
                    elif label.item() == 2:
                        perforated_total += 1
                        if pred.item() == 2:
                            perforated += 1
                except:
                    print('index error')  # 아직 인덱스를 동적으로 처리하지 못해서 트라이 캐치로 커버함
    print('Accuracy of clean : {}%'.format(100 * clean / clean_total))
    print('Accuracy of cracked : {}%'.format(100 * cracked / cracked_total))
    print('Accuracy of perforated : {}%'.format(100 * perforated / perforated_total))
    print('Accuracy of test images: %d %%' % (100 * correct / total))
    
    

    

clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road
clean_road

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1,    24] loss: 0.0243457812
[2,    24] loss: 0.0243460197
[3,    24] loss: 0.0199522513
[4,    24] loss: 0.0187673334
[5,    24] loss: 0.0180053377
[6,    24] loss: 0.0171073646
[7,    24] loss: 0.0169651890
[8,    24] loss: 0.0165712131
[9,    24] loss: 0.0165837182
[10,    24] loss: 0.0173584562
[11,    24] loss: 0.0147077402
[12,    24] loss: 0.0181985097
[13,    24] loss: 0.0169327059
[14,    24] loss: 0.0153823912
[15,    24] loss: 0.0144407312
[16,    24] loss: 0.0141003569
[17,    24] loss: 0.0140082286
[18,    24] loss: 0.0146901595
[19,    24] loss: 0.0131801275
[20,    24] loss: 0.0151132638
[21,    24] loss: 0.0136463716
[22,    24] loss: 0.0132761980
[23,    24] loss: 0.0130903818
[24,    24] loss: 0.0114123375
[25,    24] loss: 0.0116752827
[26,    24] loss: 0.0129679841
[27,    24] loss: 0.0123970951
[28,    24] loss: 0.0122980612
[29,    24] loss: 0.0113801727
[30,    24] loss: 0.0111967082
[31,    24] loss: 0.0103677672
[32,    24] loss: 0.0120458026
[33,    24] loss: